# Data Transformation and Loading Backlog

**Goal:** 
 * To transform our data by removing unecessary features as well as converting data types and loading the dataset into a platform easily accessible for our analysts to perform their analysis.

**Solution:**
 * AWS Glue ETL Jobs

### Transformation Steps Executed in Extraction:

- During our extraction phase we did some transformation with AWS lambdas we used to source our data. The lambdas get rid of duplicate entries and flatten our JSON data into a CSV file format.
- The data was flattened in Lambda because we were not able to succesfully flatten the data using AWS Glue but the result is that we saved in computing costs that are much higher for ETL jobs in AWS Glue than in AWS Lambda.

##### Extraction script used prior to transformation:
*(For demonstration purposes the script is shown without AWS libraries. The script including AWS libraries can be found in the `extraction.ipynb` backlog)*

In [1]:
import pandas as pd
import requests


url = 'https://data.cityofnewyork.us/resource/kpav-sd4t.json'


params = {'$limit': 6000}

response = requests.get(url, params=params)
data = response.json()


df_main = pd.DataFrame(data)
df_main.drop_duplicates(inplace=True)

df_main.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5458 entries, 0 to 5479
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   job_id                         5458 non-null   object
 1   agency                         5458 non-null   object
 2   posting_type                   5458 non-null   object
 3   number_of_positions            5458 non-null   object
 4   business_title                 5458 non-null   object
 5   civil_service_title            5458 non-null   object
 6   title_classification           5458 non-null   object
 7   title_code_no                  5458 non-null   object
 8   level                          5458 non-null   object
 9   job_category                   5458 non-null   object
 10  full_time_part_time_indicator  5338 non-null   object
 11  career_level                   5458 non-null   object
 12  salary_range_from              5458 non-null   object
 13  salary_r

### Transforming & Loading Features:

#### Feature Selection & Data Types
- After our extraction phase we're left with a dataset that includes 29 different features and all of the features are of string type.
- So we needed to get rid of unnecessary features and convert features leftover to their correct data type.
- To find features needed to be removed and the correct data types we conducted a data analysis on our local machines. The data analysis can be found in the main directory python script `cleaning_code_FULL.py`.

#### Configuring AWS Services:
- After we selected the features we will be using and their datatypes needed we were ready to start our AWS Glue ETL Job.
- In order for our ETL Job to work we had to setup and configure the following services in our AWS console:
    1. Redshift Namespace and Workgroup:
       * The platform our transfomred data is loaded to.
    3. IAM Role for AWS Glue permissions:
       * Permission for Glue to have read access to our extracted data stored in our S3 bucket.
       * Permission to access redshift data to execute query statements
       * Permission to access the VPC where our services were run.
    4. AWS Glue Connector:
       * We needed to create a connector in order to connect to our Redshift database.
    5. A route table for our VPC's security group.
- Our biggest challenge was trying to connect without having the proper route table set up for our VPC's security group. We were not able to succesfully run our ETL Job until we discovered that this was the missing link.


#### Creating & Running ETL Job:
- After everything was correctly configured transforming and loading our data was relatively easy.
- We ended up using Visual ETL in AWS Glue ETL Jobs which allows you to construct a blueprint for your pipeline by connecting and syncing to your configured services in real time.
- After the blueprint is constructed the script for running the Job is created and we just had to run our Job to transform and load our data.

#### ETL Job Blueprints:

## Layout of ETL Job:
![Glue Blueprint](./static/glueprint.png)
## Loading Data From S3:
![ETL 1](./static/ETL1.png)
## Dropping Tables and Converting Datatypes in Schema:
![ETL 2](./static/ETL2.png)
## Creating/Updating Table in our Redshift Database:
![ETL 3](./static/ETL3.png)
## Confirming our Job has Run Successfully:
![ETL 4](./static/ETL4.png)
## Querying our Data in Redshift:
![transformed data](./static/transformed_data.png)

## Final ETL Job Script:

In [ ]:
# import sys
# from awsglue.transforms import *
# from awsglue.utils import getResolvedOptions
# from pyspark.context import SparkContext
# from awsglue.context import GlueContext
# from awsglue.job import Job
# from awsglue import DynamicFrame

# args = getResolvedOptions(sys.argv, ["JOB_NAME"])
# sc = SparkContext()
# glueContext = GlueContext(sc)
# spark = glueContext.spark_session
# job = Job(glueContext)
# job.init(args["JOB_NAME"], args)

# # Script generated for node Amazon S3
# AmazonS3_node1708543405168 = glueContext.create_dynamic_frame.from_options(
#     format_options={
#         "quoteChar": '"',
#         "withHeader": True,
#         "separator": ",",
#         "optimizePerformance": False,
#     },
#     connection_type="s3",
#     format="csv",
#     connection_options={"paths": ["s3://semistructuredata"]},
#     transformation_ctx="AmazonS3_node1708543405168",
# )

# # Script generated for node Change Schema
# ChangeSchema_node1708543471726 = ApplyMapping.apply(
#     frame=AmazonS3_node1708543405168,
#     mappings=[
#         ("job_id", "string", "job_id", "string"),
#         ("agency", "string", "agency", "string"),
#         ("posting_type", "string", "posting_type", "string"),
#         ("number_of_positions", "string", "number_of_positions", "int"),
#         ("business_title", "string", "business_title", "string"),
#         ("civil_service_title", "string", "civil_service_title", "string"),
#         ("level", "string", "level", "string"),
#         (
#             "full_time_part_time_indicator",
#             "string",
#             "full_time_part_time_indicator",
#             "string",
#         ),
#         ("career_level", "string", "career_level", "string"),
#         ("salary_range_from", "string", "salary_range_from", "decimal"),
#         ("salary_range_to", "string", "salary_range_to", "decimal"),
#         ("salary_frequency", "string", "salary_frequency", "string"),
#         ("work_location", "string", "work_location", "string"),
#         ("division_work_unit", "string", "division_work_unit", "string"),
#         ("posting_date", "string", "posting_date", "date"),
#         ("post_until", "string", "post_until", "string"),
#     ],
#     transformation_ctx="ChangeSchema_node1708543471726",
# )

# # Script generated for node Amazon Redshift
# AmazonRedshift_node1708544284158 = glueContext.write_dynamic_frame.from_options(
#     frame=ChangeSchema_node1708543471726,
#     connection_type="redshift",
#     connection_options={
#         "redshiftTmpDir": "s3://aws-glue-assets-637423487153-us-east-1/temporary/",
#         "useConnectionProperties": "true",
#         "dbtable": "public.joblistings",
#         "connectionName": "Redshift nyc listings connect",
#         "preactions": "DROP TABLE IF EXISTS public.joblistings; CREATE TABLE IF NOT EXISTS public.joblistings (job_id VARCHAR, agency VARCHAR, posting_type VARCHAR, number_of_positions INTEGER, business_title VARCHAR, civil_service_title VARCHAR, level VARCHAR, full_time_part_time_indicator VARCHAR, career_level VARCHAR, salary_range_from DECIMAL, salary_range_to DECIMAL, salary_frequency VARCHAR, work_location VARCHAR, division_work_unit VARCHAR, posting_date DATE, post_until VARCHAR);",
#     },
#     transformation_ctx="AmazonRedshift_node1708544284158",
# )

# job.commit()

# Final Schema: 
**15 Features extracted and Converted with 5458 total current records**
- job_id VARCHAR,
- agency VARCHAR,
- posting_type VARCHAR,
- number_of_positions INTEGER,
- business_title VARCHAR,
- civil_service_title VARCHAR,
- level VARCHAR,
- full_time_part_time_indicator VARCHAR,
- career_level VARCHAR, salary_range_from DECIMAL,
- salary_range_to DECIMAL,
- salary_frequency VARCHAR,
- work_location VARCHAR, 
- division_work_unit VARCHAR, 
- posting_date DATE, 
- post_until VARCHAR